In [25]:
import pandas as pd
import os
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

path = os.path.join(current_dir, 'dev/projects/metaGclub/res/Source_Data_24Oct2022.xlsx')
print(f"Current Directory: {path}")



Current Directory: /Users/eliashanelt
Current Directory: /Users/eliashanelt/dev/projects/metaGclub/res/Source_Data_24Oct2022.xlsx


In [26]:
# Read all sheets
dfs = pd.read_excel(path, sheet_name=None)

# Now dfs is a dictionary where the keys are the sheet names
for sheet_name, df in dfs.items():
    print(f"Sheet name: {sheet_name}")
    print(df.head())

Sheet name: Directory
      Directory  \
0           NaN   
1           NaN   
2  Figure/Table   
3      Figure 1   
4      Figure 2   

  Outline of all Figures and Tables and the Source Data (tab in this file) to generate them.  \
0  "X" in columns D-I denotes data that were used...                                           
1                                                NaN                                           
2                                              Panel                                           
3                                              a - e                                           
4                                                NaN                                           

                                          Unnamed: 2        Unnamed: 3  \
0                                                NaN               NaN   
1                                                NaN               NaN   
2                                  Title/Description  subject_me